In [2]:
!ls

face.zip  sample_data


In [3]:
!unzip -q face.zip -d dataset


In [6]:
!pip install -q scikit-learn


In [7]:
import os

print("Level 1:", os.listdir("/content"))
print("Level 2 (dataset):", os.listdir("/content/dataset"))
print("Level 3 (processed_data):", os.listdir("/content/dataset/processed_data"))


Level 1: ['.config', 'dataset', 'face.zip', 'sample_data']
Level 2 (dataset): ['processed_data']
Level 3 (processed_data): ['sad', 'happy', 'fear', 'neutral', 'disgust', 'angry', 'surprise']


In [8]:
import os
import cv2
import numpy as np

# ✅ POINT HERE: your expression folders live inside processed_data
DATASET_DIR = "/content/dataset/processed_data"
IMG_SIZE = 64

X = []
y = []

# Get expression folder names (happy, sad, etc.)
expressions = sorted([
    d for d in os.listdir(DATASET_DIR)
    if os.path.isdir(os.path.join(DATASET_DIR, d))
])

print("Expressions (class order):", expressions)

for label_idx, expression in enumerate(expressions):
    folder_path = os.path.join(DATASET_DIR, expression)
    print(f"Loading {expression} from {folder_path} ...")

    for file in os.listdir(folder_path):
        img_path = os.path.join(folder_path, file)

        img = cv2.imread(img_path)
        if img is None:
            # skip broken/unreadable files
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

        X.append(img)
        y.append(label_idx)

X = np.array(X, dtype="float32")
y = np.array(y, dtype="int32")

print("X shape (before reshape):", X.shape)
print("y shape:", y.shape)


Expressions (class order): ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
Loading angry from /content/dataset/processed_data/angry ...
Loading disgust from /content/dataset/processed_data/disgust ...
Loading fear from /content/dataset/processed_data/fear ...
Loading happy from /content/dataset/processed_data/happy ...
Loading neutral from /content/dataset/processed_data/neutral ...
Loading sad from /content/dataset/processed_data/sad ...
Loading surprise from /content/dataset/processed_data/surprise ...
X shape (before reshape): (49779, 64, 64)
y shape: (49779,)


In [9]:
from tensorflow.keras.utils import to_categorical

# Add channel dimension and normalize
X = X.reshape(-1, IMG_SIZE, IMG_SIZE, 1) / 255.0

num_classes = len(expressions)
y_cat = to_categorical(y, num_classes=num_classes)

print("X shape (after reshape):", X.shape)
print("y_cat shape:", y_cat.shape)


X shape (after reshape): (49779, 64, 64, 1)
y_cat shape: (49779, 7)


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y_cat,
    test_size=0.2,
    random_state=42,
    stratify=y  # keeps class distribution similar
)

print("Train shape:", X_train.shape, y_train.shape)
print("Val shape:", X_val.shape, y_val.shape)


Train shape: (39823, 64, 64, 1) (39823, 7)
Val shape: (9956, 64, 64, 1) (9956, 7)


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 683,527 (2.61 MB)

 Trainable params: 683,527 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=15,
    batch_size=32
)


Epoch 1/15
1245/1245 ━━━━━━━━━━━━━━━━━━━━ 273s 218ms/step - accuracy: 0.2876 - loss: 1.7970 - val_accuracy: 0.5006 - val_loss: 1.3148
Epoch 2/15
1245/1245 ━━━━━━━━━━━━━━━━━━━━ 269s 216ms/step - accuracy: 0.4987 - loss: 1.3335 - val_accuracy: 0.5569 - val_loss: 1.1530
Epoch 3/15
1245/1245 ━━━━━━━━━━━━━━━━━━━━ 269s 216ms/step - accuracy: 0.5383 - loss: 1.2220 - val_accuracy: 0.5784 - val_loss: 1.1071
Epoch 4/15
1245/1245 ━━━━━━━━━━━━━━━━━━━━ 269s 216ms/step - accuracy: 0.5622 - loss: 1.1536 - val_accuracy: 0.6031 - val_loss: 1.0601
Epoch 5/15
1245/1245 ━━━━━━━━━━━━━━━━━━━━ 324s 218ms/step - accuracy: 0.5831 - loss: 1.0952 - val_accuracy: 0.5889 - val_loss: 1.1005
Epoch 6/15
1245/1245 ━━━━━━━━━━━━━━━━━━━━ 271s 218ms/step - accuracy: 0.6074 - loss: 1.0388 - val_accuracy: 0.6178 - val_loss: 1.0172
Epoch 7/15
1245/1245 ━━━━━━━━━━━━━━━━━━━━ 275s 221ms/step - accuracy: 0.6244 - loss: 0.9849 - val_accuracy: 0.6223 - val_loss: 1.0104
Epoch 8/15
1245/1245 ━━━━━━━━━━━━━━━━━━━━ 269s 216ms/step - ac

In [14]:
# 💾 Save model
model.save("expression_model.h5")
print("Model saved as expression_model.h5")

# 💾 Save class names (expressions)
with open("expressions_labels.txt", "w") as f:
    for expr in expressions:
        f.write(expr + "\n")

print("Class labels saved in expressions_labels.txt")


Model saved as expression_model.h5
Class labels saved in expressions_labels.txt


In [15]:
from google.colab import files

files.download("expression_model.h5")
files.download("expressions_labels.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
import numpy as np
import matplotlib.pyplot as plt

# Class counts
unique, counts = np.unique(y, return_counts=True)
for label_idx, count in zip(unique, counts):
    print(expressions[label_idx], ":", count)


angry : 5920
disgust : 5920
fear : 5920
happy : 11398
neutral : 8166
sad : 6535
surprise : 5920
